In [2]:
import requests

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

In [3]:
results = requests.get("https://api.data.niaid.nih.gov/v1/query?=&q=*pmid*&extra_filter=&facet_size=1000&size=100&from=0&sort=_score&use_metadata_score=true&dotfield=false&fetch_all=true&page=66")

In [4]:
data = results.json()
pmids = [hits['citation'] for hits in data['hits']]

In [5]:
pmids = []
for hits in data['hits']:
    if type(hits['citation']) is list:
        pmids.append(hits['citation'][0]['pmid'])

pmids = np.unique(pmids)

In [8]:
print(f"Gathered {len(pmids)} unique PMIDs")

Gathered 896 unique PMIDs


## Getting MeSH Terms

In [9]:
from Bio import Entrez

Entrez.email = "zqazi@scripps.edu"

dataset = pd.DataFrame(columns=['PMID', 'Abstract', 'MeSH Terms'])

def get_mesh_terms(pmid):

    try:
        # Fetch article details using the PMID
        handle = Entrez.efetch(db="pubmed", id=pmid, retmode="xml")
        article_data = Entrez.read(handle)
        handle.close()

        # Extract MeSH terms from the article data
        mesh_terms = []
        abstract = None
        if "PubmedArticle" in article_data:
            for article in article_data["PubmedArticle"]:
                if "MeshHeadingList" in article["MedlineCitation"]:
                    mesh_headings = article["MedlineCitation"]["MeshHeadingList"]
                    for heading in mesh_headings:
                        descriptor_name = heading["DescriptorName"]
                        mesh_terms.append(descriptor_name)
                    
                    # Get abstract if there are MeSH terms
                    if "Abstract" in article["MedlineCitation"]['Article']:
                        abstract = article["MedlineCitation"]['Article']["Abstract"]["AbstractText"][0]

        return abstract, mesh_terms

    except Exception as e:
        print(f"Error: {e}")
        return None


for idx, pmid_to_query in enumerate(tqdm(pmids)):

    # Get MeSH terms for the specified PMID
    abstract, mesh_terms = get_mesh_terms(pmid_to_query)

    if mesh_terms and abstract:
        # print(f"MeSH terms for PMID {pmid_to_query}:\n")
        # for term in mesh_terms_result:
        #     print(term)
        
        # {'PMID': pmid_to_query, 'Abstract': str(abstract), 'MeSH Terms': [str(term) for term in mesh_terms]}
        dataset.loc[len(dataset.index)] = [pmid_to_query, abstract, [str(term) for term in mesh_terms]]

    if not mesh_terms and not abstract:
        print(f"Failed to retrieve MeSH terms AND Abstract for PMID {pmid_to_query}.")
    elif not mesh_terms:
        print(f"Failed to retrieve MeSH terms for PMID {pmid_to_query}.")
    elif not abstract:
        print(f"Failed to retrieve Abstract for PMID {pmid_to_query}.")
    


  0%|          | 0/896 [00:00<?, ?it/s]

Failed to retrieve MeSH terms for PMID 18700954.
Failed to retrieve Abstract for PMID 18700954.
Failed to retrieve MeSH terms for PMID 19505295.
Failed to retrieve Abstract for PMID 19505295.
Failed to retrieve MeSH terms for PMID 20148099.
Failed to retrieve Abstract for PMID 20148099.
Failed to retrieve MeSH terms for PMID 23178508.
Failed to retrieve Abstract for PMID 23178508.
Failed to retrieve MeSH terms for PMID 28868355.
Failed to retrieve Abstract for PMID 28868355.
Failed to retrieve MeSH terms for PMID 29326929.
Failed to retrieve Abstract for PMID 29326929.
Failed to retrieve MeSH terms for PMID 29675033.
Failed to retrieve Abstract for PMID 29675033.
Failed to retrieve Abstract for PMID 29778662.
Failed to retrieve MeSH terms for PMID 29844930.
Failed to retrieve Abstract for PMID 29844930.
Failed to retrieve MeSH terms for PMID 30258415.
Failed to retrieve Abstract for PMID 30258415.
Failed to retrieve MeSH terms for PMID 30352973.
Failed to retrieve Abstract for PMID 303

In [10]:
dataset.head()

,PMID,Abstract,MeSH Terms
0,12524544,Nuclear export of mRNA is mediated by a comple...,"[Active Transport, Cell Nucleus, DNA-Binding P..."
1,14512514,"The synthesis of fatty acids and cholesterol, ...","[Animals, CCAAT-Enhancer-Binding Proteins, Cho..."
2,14981237,"RNase E, an essential endoribonuclease of Esch...","[Bacterial Proteins, Endoribonucleases, Escher..."
3,15054141,Diminished activity of peroxisome proliferator...,"[Animals, Aorta, Gene Expression Profiling, Ge..."
4,15129285,The flood of high-throughput biological data h...,"[Aerobiosis, Anaerobiosis, Computational Biolo..."


## Mapping MeSH terms to EDAM ontology

First, a proof of concept...

In [11]:
import text2term

/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [12]:
edam_ontology = text2term.cache_ontology("https://data.bioontology.org/ontologies/EDAM/submissions/44/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb", "EDAM")

2023-11-17 12:53:13 INFO [text2term.term_collector]: Loading ontology https://data.bioontology.org/ontologies/EDAM/submissions/44/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb...
2023-11-17 12:53:15 INFO [text2term.term_collector]: ...done (ontology loading time: 2.28s)
2023-11-17 12:53:15 INFO [text2term.term_collector]: Collecting ontology term details...
2023-11-17 12:53:15 INFO [text2term.term_collector]: ...done: collected 3577 ontology terms (collection time: 0.11s)
2023-11-17 12:53:15 INFO [text2term.t2t]: Filtered ontology terms to those of type: OntologyTermType.ANY
2023-11-17 12:53:15 INFO [text2term.t2t]: Caching ontology https://data.bioontology.org/ontologies/EDAM/submissions/44/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb to: cache/EDAM


In [13]:
mapped_terms = text2term.map_terms([str(term) for term in dataset.iloc[0]['MeSH Terms']], "EDAM", use_cache=True)
mapped_terms[['Source Term', 'Mapped Term Label']].head()

2023-11-17 12:53:17 INFO [text2term.t2t]: Loading cached ontology from: cache/EDAM/EDAM-term-details.pickle
2023-11-17 12:53:17 INFO [text2term.t2t]: Filtered ontology terms to those of type: OntologyTermType.CLASS
2023-11-17 12:53:17 INFO [text2term.t2t]: Mapping 20 source terms to EDAM
2023-11-17 12:53:21 INFO [text2term.t2t]: ...done (mapping time: 3.45s seconds)


,Source Term,Mapped Term Label
0,"Active Transport, Cell Nucleus",Cell line report
1,"Active Transport, Cell Nucleus",Nucleic acid report
2,"Active Transport, Cell Nucleus",Active site prediction
3,DNA-Binding Proteins,DNA binding sites
4,DNA-Binding Proteins,DNA binding site prediction


### Map All MeSH Terms

In [14]:
def convert_mesh_to_edam(mesh_terms):
    mapped_terms = text2term.map_terms(mesh_terms, "EDAM", use_cache=True)

    return mapped_terms['Mapped Term Label'].to_list()

dataset['EDAM Topics'] = dataset['MeSH Terms'].apply(lambda terms: convert_mesh_to_edam(terms))

2023-11-17 12:53:23 INFO [text2term.t2t]: Loading cached ontology from: cache/EDAM/EDAM-term-details.pickle
2023-11-17 12:53:23 INFO [text2term.t2t]: Filtered ontology terms to those of type: OntologyTermType.CLASS
2023-11-17 12:53:23 INFO [text2term.t2t]: Mapping 20 source terms to EDAM
2023-11-17 12:53:23 INFO [text2term.t2t]: ...done (mapping time: 0.09s seconds)
2023-11-17 12:53:23 INFO [text2term.t2t]: Loading cached ontology from: cache/EDAM/EDAM-term-details.pickle
2023-11-17 12:53:23 INFO [text2term.t2t]: Filtered ontology terms to those of type: OntologyTermType.CLASS
2023-11-17 12:53:23 INFO [text2term.t2t]: Mapping 17 source terms to EDAM
2023-11-17 12:53:23 INFO [text2term.t2t]: ...done (mapping time: 0.07s seconds)
2023-11-17 12:53:23 INFO [text2term.t2t]: Loading cached ontology from: cache/EDAM/EDAM-term-details.pickle
2023-11-17 12:53:23 INFO [text2term.t2t]: Filtered ontology terms to those of type: OntologyTermType.CLASS
2023-11-17 12:53:23 INFO [text2term.t2t]: Mappi

In [15]:
dataset[['MeSH Terms', 'EDAM Topics']]

,MeSH Terms,EDAM Topics
0,"[Active Transport, Cell Nucleus, DNA-Binding P...","[Cell line report, Nucleic acid report, Active..."
1,"[Animals, CCAAT-Enhancer-Binding Proteins, Cho...","[Zoology, Laboratory animal science, Animal st..."
2,"[Bacterial Proteins, Endoribonucleases, Escher...","[Proteins, protein, Protein-protein interactio..."
3,"[Animals, Aorta, Gene Expression Profiling, Ge...","[Zoology, Laboratory animal science, Animal st..."
4,"[Aerobiosis, Anaerobiosis, Computational Biolo...","[Analysis, Protein analysis, Sequence analysis..."
...,...,...
810,"[Astrocytes, Humans, Proteomics, Sleep, Sleep ...","[Human biology, Human genetics, Human disease,..."
811,"[Cell Cycle Proteins, Chromatin, DNA Damage, D...","[Cell cycle, Proteins, protein, Immunoprecipit..."
812,"[Mice, Animals, Huntington Disease, Acetylatio...","[Mice or rats, Rice, Zoology, Laboratory anima..."
813,"[Humans, NAD, Sequestosome-1 Protein, Antioxid...","[Human biology, Human genetics, Human disease,..."


In [16]:
print("Any unmapped MeSH Terms?: ", dataset['EDAM Topics'].isna().any())

Any unmapped MeSH Terms?:  False


## Save Testing Set

In [17]:
print("Check for N/A values:")

dataset.isna().any()

Check for N/A values:


PMID           False
Abstract       False
MeSH Terms     False
EDAM Topics    False
dtype: bool

In [18]:
print("Number of entries: ", len(dataset))

Number of entries:  815


In [19]:
dataset.to_csv(input("Enter file path and name: "))